# Redes Neuronales
## TP1

Entrene una red de Hopfield ‘82 con las imágenes binarias disponibles en el campus.
1. Verifique si la red aprendió las imágenes enseñadas.
2. Evalúe la evolución de la red al presentarle versiones alteradas de las imágenes aprendidas: agregado de ruido, elementos borrados o agregados.
3. Evalúe la existencia de estados espurios en la red: patrones inversos y combinaciones de un número impar de patrones. (Ver Spurious States, en la sección 2.2, Hertz, Krogh & Palmer, pág. 24).
4. Realice un entrenamiento con las 6 imágenes disponibles. ¿Es capaz la red de aprender todas las imágenes? Explique.

In [ ]:
# primero importamos numpy y algo para leer imágenes

import numpy as np
from PIL import Image

In [ ]:
# necesito paths a las imagenes,vienen de un repo clonado localmente 
path1 = 'C:\\Users\\ignac\\Documents\\GitHub\\Redes-neuronales\\imagenes_tp1\\paloma.bmp'
path2 = 'C:\\Users\\ignac\\Documents\\GitHub\\Redes-neuronales\\imagenes_tp1\\panda.bmp' # 50*50
path3 = 'C:\\Users\\ignac\\Documents\\GitHub\\Redes-neuronales\\imagenes_tp1\\perro.bmp' #50*50
path4= 'C:\\Users\\ignac\\Documents\\GitHub\\Redes-neuronales\\imagenes_tp1\\quijote.bmp'
path5= 'C:\\Users\\ignac\\Documents\\GitHub\\Redes-neuronales\\imagenes_tp1\\torero.bmp'
path6= 'C:\\Users\\ignac\\Documents\\GitHub\\Redes-neuronales\\imagenes_tp1\\v.bmp' #50*50


In [ ]:
def imgPath_to_vector(paths):
    """La idea es tomar una lista de paths ygenerar una matriz cuyas columnas sean los pixeles de las imagenes"""
    list = []
    for path in paths:
        img = Image.open(path) 
        vector_img = np.array(img).astype(np.int8).ravel()#ideal del profe, tratar las imagenes como vectores columna
        list.append(vector_img *2 -1) # acá lo ajusto para ir de (0,1)->(-1,1)
    return np.asarray(list).transpose()



In [40]:
flattened_images1 = imgPath_to_vector([path2,path3,path6]) # tenemos las imagenes 

In [54]:
print(flattened_images1)

[[ 1 -1 -1]
 [ 1 -1 -1]
 [ 1 -1 -1]
 ...
 [ 1 -1 -1]
 [ 1 -1 -1]
 [ 1 -1 -1]]


In [60]:
np.asmatrix(flattened_images1[:,0]).T

matrix([[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]], dtype=int8)

In [68]:
# ahora sabemos que hay tantas neuronas como valores en los vectores columna que conforman la matriz de datos "flattened_images1"
# W es una matriz cuadrada de filas y columnas de igual tamaño a la cantidad de neuronas

filas = np.shape(flattened_images1)[0]

W_matrix = np.zeros([filas,filas]) # creo la matriz W,de pesos

for x in range(10):
    delta_W = np.asmatrix(flattened_images1[:,0]).T @ np.asmatrix(flattened_images1[:,0])
    W_matrix = W_matrix + delta_W





In [ ]:
print(W_matrix.tolist())